# 00 - Extraccion de datos mediante API de Mercado Libre

## Importación de librerias

In [1]:
import pandas as pd
import numpy as np
import json
import requests

from datetime import datetime, timedelta, timezone, date
pd.options.display.max_columns=None

try:
    from pyprojroot import here
    print("Libreria: 'pyprojroot' esta instalada y se cargo correctamente")
except ModuleNotFoundError:
    print("Libreria: 'pyprojroot' no esta instalada, se debe instalar")
    !pip install pyprojroot

import matplotlib.pyplot as plt
import os

try:
    import seaborn as sns
    print("Libreria: 'seaborn' esta instalada y se cargo correctamente")
except ModuleNotFoundError:
    print("Libreria: 'seaborn' no esta instalada, se debe instalar")
    !pip install seaborn


Libreria: 'pyprojroot' esta instalada y se cargo correctamente
Libreria: 'seaborn' esta instalada y se cargo correctamente


In [2]:
here()

WindowsPath('C:/Users/marti/OneDrive/Escritorio/Tesis/mdc-tesis')

In [3]:
path_save = here() / "data" 

In [4]:
import sys
sys.path.append(here())
from utils.utils import get_q_items, create_item_list, get_available_filters, get_df_list

## Armado del Dataset

Para armar el dataset descargaremos para Mercado Libre Argentina laptops publicadas. La categoría se llama: MLA1652.
La API tiene una restricción que no permite utilizar un offset mayor a 4000. Esto nos permite descargar menos de 4000 publicaciones por loop.
Es por ello que se han armado ciertos grupos para descargar la información por grupo. Esto nos permite tambien etiquetar a las publicaciones con este atributo, 
elemento que no se obtiene descargando la información de la publicación.

<img src="Grupos.png" alt="Drawing" style="width: 501px;"/>


Para armar estos grupos necesitamos filtar desde el url mediante los siguientes nombres:  

* installments:  
    + yes  
    + no_interest  
* power_seller:  
    + yes  
    + no  
* shipping_cost:  
    + get_available_filtersfree  
            

### Grupo 1

In [5]:
filters = {"installments":"yes",
           "power_seller":"yes",
          "shipping_cost":"free"}


In [6]:
maximum = get_q_items(filters = filters)
print(f'Encontramos {maximum} resultados para nuestra consulta')


http://auth.mercadolibre.com.ar/authorization?response_type=code&client_id=8800815426780367&redirect_uri=https://www.google.com.ar/


Codigo : TG-6303ef96351783000185c312-216145475


<Response [200]>
Encontramos 5492 resultados para nuestra consulta


In [7]:
item_list = create_item_list(filters = filters)

El offst es 4000 se terminará la ejecución.


In [8]:
data = get_df_list(item_list)
data["installment"] = "yes"
data["power_seller"] = "yes" 


data["shipping_cost"] = "free"

### Grupo 2

In [9]:
filters = {"installments":"yes",
           "power_seller":"no",
          "shipping_cost":"free"}


In [10]:
maximum = get_q_items(filters = filters)
print(f'Encontramos {maximum} resultados para nuestra consulta')


Encontramos 13958 resultados para nuestra consulta


In [11]:
item_list = create_item_list(filters = filters)

El offst es 4000 se terminará la ejecución.


In [12]:
data2 = get_df_list(item_list)
data2["installment"] = "yes"
data2["power_seller"] = "no"
data2["shipping_cost"] = "free"

### Grupo 3

In [13]:
filters = {"installments":"no_interest",
           "power_seller":"yes",
          "shipping_cost":"free"}


In [14]:
maximum = get_q_items(filters = filters)
print(f'Encontramos {maximum} resultados para nuestra consulta')


Encontramos 3519 resultados para nuestra consulta


In [15]:
item_list = create_item_list(filters = filters)

In [16]:
data3 = get_df_list(item_list)
data3["installment"] = "no_interest"
data3["power_seller"] = "yes"
data3["shipping_cost"] = "free"

### Grupo 4

In [17]:
filters = {"installments":"no_interest",
           "power_seller":"no",
          "shipping_cost":"free"}


In [18]:
maximum = get_q_items(filters = filters)
print(f'Encontramos {maximum} resultados para nuestra consulta')


Encontramos 5069 resultados para nuestra consulta


In [19]:
item_list = create_item_list(filters = filters)

El offst es 4000 se terminará la ejecución.


In [20]:
data4 = get_df_list(item_list)
data4["installment"] = "no_interest"
data4["power_seller"] = "no"
data4["shipping_cost"] = "free"

________________________

In [21]:
data = pd.concat([data, data2, data3, data4])

In [22]:
data.shape

(15493, 76)

In [23]:
data.title.value_counts()

Notebook Asus Vivobook Core I3 11va 8gb 512gb 17 Pulgadas       16
Notebook Asus R410ma Intel Celeron 4gb 128gb Emmc Windows 10    14
Notebook Pcbox Fire 3 I3-1005g1 8gb Ram  Ssd 256 14  Win10      12
Apple Macbook Pro 13 Touch Bar M1 8gb 256gb Space Gray          10
Notebook Hp Amd Ryzen 5 8gb Ram 256gb Ssd Windows 11 Full Hd    10
                                                                ..
Notebook Lenovo Core I5 ( 16gb + 256gb Ssd ) Qhd W10 Outlet      1
Notebook Positivo Bgh Z100 Cel 2.16 Ghz/ssd 120 Gb /4 Gb Ram     1
Notebook 16gb + 256 Ssd / Hp 13 Ryzen 7 Fhd Win 10 Outlet        1
Notebook Hp Pavilion X360 14-dy0502la Core I3 8gb 256 M2 W10     1
Macbook Air 2015 13 Usada                                        1
Name: title, Length: 6680, dtype: int64

In [24]:
data.to_csv(path_save / "datos_laptops.csv", index = False, sep = ";")